## Explanatory Notebook for the Bot Classes

The code below tries to add explanation to the various bot classes defined below

The Following code is ispired from the [Dialog Self-Play](https://arxiv.org/abs/1801.04871) Paper from google.

Below I have tried to add as much explanation to the cells as I could. However if you feel anything is confusing then feel free to drop a main to me at smajumdar@fbk.eu

**Importing Libraries**

In [1]:
import random

# the libraries imported below serve the function of importing the Action Class defined one directory structure above
import sys
sys.path.append("..")
from utils import Action

### The Bot classes

**Who is the Bot Class ?**

The Bot class can be thought to immitiate the _the system agent_ (in this case, the employee dealing in transactions).

**What is this complex piece of code below , can I understand it ?**

At first glance, No. But if it would help what it represents. The Code below is nothing more than a Finite State Machine.

**Okay !! Explain how it is working ?**

The _Bot_ or the FSM is working through a pre-determined set of states where at each state we assume that the bot will perform an action related to the field. May be the diagram below will help.

**What is next_state and bot_action ?**

So Basically, for each of the user's action we determine what is the appropriate *bot\_action* to be performed. This done with the help of the speak function and appropriate state function.

Once we have determined this, we determine what should be the appropriate next_state in the diagram.

If you look at the speak function it selects the appropriate function (or next_state of the diagram) from the set of states

**Creating Transaction Bot**

This bot converses and requests queries - through actions - in the transaction domain.

In [2]:
class Transaction_bot(object) :
    
    def __init__(self) :
        
        self.last_slot = None
        self.list_of_slots = ["user_account","destination_name","amount"]
        self.slots_to_ask = ["user_account","destination_name","amount"]
        self.user_values = dict()
        self.states = {"initial" : self.initial_state ,
                       "check_initial" : self.check_initial_state,
                       "user_account" : self.account_state ,
                       "check_account" : self.check_account_state,
                       "change_account" : self.change_account_state,
                       "destination_name" : self.destination_state ,
                       "check_destination" : self.check_destination_name_state,
                       "change_destination_name" : self.change_destination_name_state,
                       "amount" : self.amount_state ,
                       "check_amount" : self.check_amount_state,
                       "change_amount" : self.change_amount_state,
                       "end_call" : self.end_call_state ,
                       "balance" : self.balance_state ,
                       "confirmation_state" : self.confirmation_state,
                       "api_call" : self.api_call_state}
        
        self.priority_states = list()
        self.priority_actions = dict()
        
        self.current_state = self.initial_state
    
    def sort_my_slots(self,slots_given) :
        slots_sorted = list()
        
        if "user_account" in slots_given :
            slots_sorted.append("user_account")
            slots_given.remove("user_account")
        
        if "destination_name" in slots_given :
            slots_sorted.append("destination_name")
            slots_given.remove("destination_name")
        
        if "amount" in slots_given :
            slots_sorted.append("amount")
            slots_given.remove("amount")
        
        for slot in slots_given :
            slots_sorted.append(slot)
        
        return slots_sorted
    
    # store any new values given by the user
    def record_user_values(self,user_action) :
        
        if type(user_action.get_values()) == dict :
            
            for slot,values in user_action.get_values().items() :
                self.user_values[slot] = values
                
     # remove the slots given from the list of slots to ask           
    def remove_informed_slots(self,user_action) :
        
        for slot in user_action.get_slots() :
            
            if slot in self.slots_to_ask :
                
                self.slots_to_ask.remove(slot)
                
    def speak(self,user_action) :
        
        if user_action == None :
            
            print("user_action received is None")
        
        next_state , bot_action = self.current_state(user_action)
        self.current_state = self.states[next_state]
        
        return bot_action
        
    # meet the initial state, here the user may provide one or more than one values
    def initial_state(self,user_action) :
        
        self.record_user_values(user_action)
        self.remove_informed_slots(user_action)
        
        if user_action.get_slots() :
            
            if "intent" in user_action.get_slots() and len(user_action.get_slots()) > 1 :
                
                next_state = "check_initial"
                slots_given = user_action.get_slots()[1:] 
                slot_message = " "
                for slot in slots_given :
                    slot_message += "{}:{},".format(slot,self.user_values[slot])
                bot_action = Action(actor="API",
                                    action="initial_slots_check",
                                    slots=user_action.get_slots(),
                                    values=None,
                                    message=slot_message[:-1])
            
            else :
                
                next_state = self.slots_to_ask[0]
                bot_action = Action(actor="Bot",
                                    action="request",
                                    slots=[next_state],
                                    values=None,
                                    message="requesting the values for {}".format(next_state))        
        else :           
            
            next_state = "initial"
            bot_action = Action(actor="Bot",
                                action="request",
                                slots=["intent"],
                                values=None,
                                message="Get the intent first")
        
        return next_state , bot_action
    
    def check_initial_state(self,user_action) :
        # if the below message is received then it means that initial check is successful and move on to the next appropriate slots
        
        if user_action.get_message() == "initial_slots_check : success" :
            
            if not self.slots_to_ask :
                
                next_state = "confirmation_state"
                bot_action = Action(actor="Bot",
                                    action="request",
                                    slots=None,
                                    values=None,
                                    message="confirm transaction ?")
            
            else :
                
                next_state = self.slots_to_ask[0]
                bot_action = Action(actor="Bot",
                                    action="request",
                                    slots=[next_state],
                                    values=None,
                                    message="request for {} ".format(next_state))
        
        else :
            
            self.priority_states = user_action.get_slots()
            self.priority_actions = user_action.get_values()
            
            next_state = self.priority_states[0]
            bot_action = self.priority_actions[next_state]
            
            self.priority_states.remove(next_state)
        
        return next_state , bot_action
    
    def account_state(self,user_action) :
        
        # if user account has been given then 
        if "user_account" in user_action.get_slots() :
            
            # remove the slot which has already been asked
            self.remove_informed_slots(user_action)
                
            # update user infomation
            user_values = user_action.get_values()
            
            # record and store all the values given by the user
            self.record_user_values(user_action)
            
            
            # pick a slot to ask randomly from the remaining slots_to_ask
            next_state = "check_account"
            
            # perform the corresponding bot information
            bot_action = Action(actor="API",
                                action="account_check",
                                slots=["user_account"],
                                values=None,
                                message="user_account:{}".format(self.user_values["user_account"]))
                    
        else :
            
            next_state = "user_account"
            bot_action = Action(actor="Bot",
                                action="request",
                                slots=["user_account"],
                                values=None,
                                message="requesting user to specify the account")
            
        return next_state , bot_action
    
    def check_account_state(self,user_action) :
        
        if user_action.get_message() == "account_check : success" :
            
            if "amount" in self.user_values.keys() :
                
                next_state = "check_amount"
                bot_action = Action(actor="API",
                                    action="amount_check",
                                    slots=["limit","balance"],
                                    values=None,
                                    message=" user_account:{} amount:{}".format(self.user_values["user_account"],self.user_values["amount"]))
            
            else :
                
                if self.priority_states :
                    next_state = self.priority_states[0]
                    bot_action = self.priority_actions[next_state]
                
                    self.priority_states.remove(next_state)
            
                elif not self.slots_to_ask :
                    
                    next_state = "confirmation_state"
                    bot_action = Action(actor="Bot",
                                        action="request",
                                        slots=None,
                                        values=None,
                                        message="confirm transaction ?")
            
                else :
                    
                    next_state = self.slots_to_ask[0]
                    bot_action = Action(actor="Bot",
                                        action="request",
                                        slots=[next_state],
                                        values=None,
                                        message="request for {} ".format(next_state))        
        else :
            
            next_state = "change_account"
            slot_message = ",".join(user_action.get_slots())
            bot_message = "It seems that you have not entered a valid account, you available accounts are {}, would you like change the source account ?".format(slot_message)
            bot_action = Action(actor="Bot",
                                action="request",
                                slots=None,
                                values=None,
                                message=bot_message,
                                description="CHANGE_ACCOUNT")
        
        return next_state , bot_action
    
    def change_account_state(self,user_action) :
        
        if user_action.get_message() == "accept" :
            
            next_state = "user_account"
            bot_action = Action(actor="Bot",
                                action="request",
                                slots=[next_state],
                                values=None,
                                message="Requesting user to provide new user account")
        
        else :
            
            next_state = "end_call"
            bot_action = Action(actor="Bot",
                                action="end_call",
                                slots=None,
                                values=None,
                                message="You denied to change the account")
        
        return next_state , bot_action 
    
    def destination_state(self,user_action) :
        # remove the slot already asked
        self.remove_informed_slots(user_action)
            
        # update the user information with the new values got
        self.record_user_values(user_action)
        
        if "destination_name" in user_action.get_slots() :
            
            # sample out a new state based on the remaining slots to ask
            next_state = "check_destination"
            bot_action = Action(actor="API",
                                action="destination_name_check",
                                slots=["destination_name"],
                                values=None,
                                message="destination_name:{}".format(self.user_values["destination_name"]))
        
        else :
            
            next_state = "destination_name"
            bot_action = Action(actor="Bot",
                                action="request",
                                slots=["destination_name"],
                                values=None,
                                message="Provide the Name of the Receiver")
        
        return next_state , bot_action
    
    def check_destination_name_state(self,user_action) :
        
        if user_action.get_message() == "destination_name_check : success" :
            
            if self.priority_states :
                next_state = self.priority_states[0]
                bot_action = self.priority_actions[next_state]
                
                self.priority_states.remove(next_state)
                
            elif not self.slots_to_ask :
                
                next_state = "confirmation_state"
                bot_action = Action(actor="Bot",
                                    action="request",
                                    slots=None,
                                    values=None,
                                    message="confirm transaction ?")
                
            else :
                next_state = self.slots_to_ask[0]
                bot_action = Action(actor="Bot",
                                    action="request",
                                    slots=[next_state],
                                    values=None,
                                    message="Requesting the name for {}".format(next_state))
        
        else :
            
            next_state = "change_destination_name"
            slot_message = ",".join(user_action.get_slots())
            bot_message = "The recipient you are trying to provide doesn't exist, available list of recipients is {}, would you like to change the recipient ?".format(slot_message)
            bot_action = Action(actor="Bot",
                                action="request",
                                slots=None,
                                values=None,
                                message=bot_message,
                                description="CHANGE_DESTINATION_NAME")
        
        return next_state , bot_action
    
    def change_destination_name_state(self,user_action) :
        
        if user_action.get_message() == "accept" :
            
            next_state = "destination_name"
            bot_action = Action(actor="Bot",
                                action="request",
                                slots=[next_state],
                                values=None,
                                message="Requesting user to provide new user account")
        
        else :
            
            next_state = "end_call"
            bot_action = Action(actor="Bot",
                                action="end_call",
                                slots=None,
                                values=None,
                                message="User failed to change the account")
        
        return next_state , bot_action
    
    def amount_state(self,user_action) :
        
        self.record_user_values(user_action)
        self.remove_informed_slots(user_action)
        
        if "amount" in user_action.get_slots() :
             
            if "user_account" in self.user_values.keys() :
                
                # No random check this time because we have to check if the amount given is correct or not
                next_state = "check_amount"
                bot_action = Action(actor="API",
                                    action="amount_check",
                                    slots=["limit","balance"],
                                    values=None,
                                    message="user_account:{} amount:{}".format(self.user_values["user_account"],self.user_values["amount"]))
            
            else :
                
                next_state = self.slots_to_ask[0]
                bot_action = Action(actor="Bot",
                                    action="request",
                                    slots=[next_state],
                                    values=None,
                                    message="requesting user to provide {} ".format(next_state))
        
        else :
            
            next_state = "amount"
            bot_action = Action(actor="Bot",
                                action="request",
                                slots=["amount"],
                                values=None,
                                message="requesting the user to provide the Amount")
            
        return next_state , bot_action

    
    def check_amount_state(self,user_action) :
        
        self.record_user_values(user_action)
        
        if user_action.get_message() == "amount_check : success" :
            
            if self.priority_states :
                next_state = self.priority_states[0]
                bot_action = self.priority_actions[next_state]
                
                self.priority_states.remove(next_state)
            
            elif not self.slots_to_ask :
                
                next_state = "confirmation_state"
                bot_action = Action(actor="Bot",
                                    action="request",
                                    slots=None,
                                    values=None,
                                    message="confirm transaction ?")
            
            else :
                
                next_state = self.slots_to_ask[0]
                bot_action = Action(actor="Bot",
                                    action="request",
                                    slots=[next_state],
                                    values=None,
                                    message="request for {} ".format(next_state))

        else :
            
            next_state = "change_amount"
            bot_action = Action(actor="Bot",
                                action="request",
                                slots=None,
                                values=None,
                                message="It seems the amount you provided can't be processed because your transaction limit is {} and your current balance is {} so the maximum you can transfer is {}, would you like to reduce your amount to this amount ?".format(self.user_values["limit"],self.user_values["balance"],self.user_values["max_transferable_amt"]),
                                description="CHANGE_TO_MAX_TRANSFERABLE_AMT")
        
        return next_state , bot_action
    
    
    def change_amount_state(self,user_action) :
        
        if user_action.get_message() == "accept" :
            
            self.user_values["amount"] = self.user_values["max_transferable_amt"]
            
            if self.priority_states :
                next_state = self.priority_states[0]
                bot_action = self.priority_actions[next_state]
                
                self.priority_states.remove(next_state)
            
            elif not self.slots_to_ask :
                
                next_state = "confirmation_state"
                bot_action = Action(actor="Bot",
                                    action="request",
                                    slots=None,
                                    values=None,
                                    message="confirm transaction ?")
            
            else :
                
                next_state = self.slots_to_ask[0]
                bot_action = Action(actor="Bot",
                                    action="request",
                                    slots=[next_state],
                                    values=None,
                                    message="request for {} ".format(next_state))
        
        else :
            
            next_state = "end_call"
            bot_action = Action(actor="Bot",
                                action="end_call",
                                slots=None,
                                values=None,
                                message="Rejected to change the decision")
        
        return next_state , bot_action
    
    # This is a dead state and I don't know why it is here , but I don't know what will happen if I remove this function
    def balance_state(self,user_action) :
        return
    
    # ask for confirmation
    def confirmation_state(self,user_action) :
        
        if user_action.get_message() == "accept" :
            
            next_state = "api_call"
            api_value = self.user_values["user_account"] + " " + self.user_values["destination_name"] + " "  + str(self.user_values["amount"])
            bot_action = Action(actor="Bot",
                                action="api_call",
                                slots=None,
                                values={"api_call" : api_value},
                                message="API_CALL",
                                description="API_CALL")
        
        else :
            
            next_state = "end_call"
            bot_action = Action(actor="Bot",
                                action="end_call",
                                slots=None,
                                values=None,
                                message="User refused to confirm transaction")
        
        return next_state, bot_action

    # end the call
    def end_call_state(self,user_action) :
        
        print("inside end_call state")
        
        return
    
    # Api call state
    def api_call_state(self,user_action) :
        
        if user_action.get_message() == "api_call success" :
            
            bot_action = Action(actor="Bot",
                                action="end_call",
                                slots=None,
                                values=None,
                                message="{} conducted successfully, ciao !!".format(self.user_values["intent"]))
        
        else :
            
            bot_action = Action(actor="Bot",
                                action="end_call",
                                slots=None,
                                values=None,
                                message="error in processing {}".format(self.user_values["intent"]))
        
        next_state = "end_call"
        
        return next_state , bot_action

**Creating Account Balance Bot**

The class below, creates a bot capable on conversing - through actions - in the Account Balance Domain.

In [3]:
class Account_bot(object) :
    
    def __init__(self) :
        
        self.last_slot = None
        self.list_of_slots = ["user_account","destination_name","amount"]
        self.slots_to_ask = ["user_account","destination_name","amount"]
        self.user_values = dict()
        self.states = {"initial" : self.initial_state ,
                       "list_accounts" : self.list_accounts_state,
                       "select_account" : self.select_account_state,
                       "api_call" : self.api_call_state,
                       "end_call" : self.end_call_state}
        
        self.priority_states = list()
        self.priority_actions = dict()
        
        self.current_state = self.initial_state
    
    def sort_my_slots(self,slots_given) :
        slots_sorted = list()
        
        if "user_account" in slots_given :
            slots_sorted.append("user_account")
            slots_given.remove("user_account")
        
        if "destination_name" in slots_given :
            slots_sorted.append("destination_name")
            slots_given.remove("destination_name")
        
        if "amount" in slots_given :
            slots_sorted.append("amount")
            slots_given.remove("amount")
        
        for slot in slots_given :
            slots_sorted.append(slot)
        
        return slots_sorted
    
    # store any new values given by the user
    def record_user_values(self,user_action) :
        
        if type(user_action.get_values()) == dict :
            
            for slot,values in user_action.get_values().items() :
                self.user_values[slot] = values
                
     # remove the slots given from the list of slots to ask           
    def remove_informed_slots(self,user_action) :
        
        if user_action.get_slots() :
            for slot in user_action.get_slots() :
                if slot in self.slots_to_ask :
                    self.slots_to_ask.remove(slot)
                
    def speak(self,user_action) :
        
        if user_action == None :
            
            print("user_action received is None")
        
        next_state , bot_action = self.current_state(user_action)
        self.current_state = self.states[next_state]
        
        return bot_action
        
    # meet the initial state, here the user may provide one or more than one values
    # request intent if not given already
    def initial_state(self,user_action) :
        self.record_user_values(user_action)
        self.remove_informed_slots(user_action)
        
        if "intent" in user_action.get_slots() :
            next_state = "list_accounts"
            bot_action = Action(actor="API",
                               action="request_accounts",
                               slots=["name"],
                               values=None,
                               message="accounts:{}".format(self.user_values["name"]))
        else :
            next_state = "initial"
            bot_action = Action(actor="Bot",
                               action="request",
                               slots=["intent"],
                               values=None,
                               message="requesting the intent from the user")
        return next_state , bot_action
    
    def list_accounts_state(self,user_action) :
        
        if user_action.get_description() == "LIST_OF_SLOTS" :
            next_state = "select_account"
            slot_message = ",".join(user_action.get_slots())
            bot_message = "You have the following accounts : {} , which one do you wish ?".format(slot_message)
            bot_action = Action(actor="Bot",
                                action="request",
                                slots=None,
                                values=None,
                                message=bot_message,
                                description="SELECT_ACCOUNT")
        else : 
            next_state = "list_accounts"
            bot_action = Action(actor="API",
                               action="request_accounts",
                               slots=["name"],
                               values=None,
                               message="accounts:{}".format(self.user_values["name"]))
            
        return next_state , bot_action
        
    def select_account_state(self,user_action) :
        
        self.record_user_values(user_action)
        self.remove_informed_slots(user_action)
        
        if "user_account" in user_action.get_slots() :
            next_state = "api_call"
            api_value = self.user_values["user_account"]
            bot_action = Action(actor="Bot",
                               action="api_call",
                               slots=None,
                               values={"api_call" : api_value},
                               message="API_CALL",
                               description="API_CALL")
        else :
            next_state = "select_account"
            bot_action = Action(actor="Bot",
                               action="request",
                               slots=None,
                               values=None,
                               message="Please select an account")
        
        return next_state , bot_action
    
    def api_call_state(self,user_action) :
        
        self.record_user_values(user_action)
        self.remove_informed_slots(user_action)
        
        if user_action.get_message().startswith("api_call : success") :
            next_state = "end_call"
            bot_message = "your current balance for account:{} is {}".format(self.user_values["user_account"],self.user_values["balance"])
            bot_action = Action(actor="Bot",
                                action="end_call",
                               slots=None,
                               values=None,
                               message=bot_message)
        else :
            next_state = "end_call"
            bot_action = Action(actor="Bot",
                               action="end_call",
                               slots=None,
                               values=None,
                               message="error in processing request !!")
        
        return next_state , bot_action
    
    def end_call_state(self,user_action) :
        print("Reached end of transaction")
        return

**Creating Transaction History Domain**

This bot creates a bot capable of conversing - through actions - in the Transaction History Domain.

In [4]:
class Transaction_history_bot(object) :
    
    def __init__(self,templates=None) :
        
        self.last_slot = None
        self.list_of_slots = ["user_account","destination_name","amount"]
        self.slots_to_ask = ["user_account","destination_name","amount"]
        self.user_values = dict()
        self.states = {"initial" : self.initial_state ,
                       "list_accounts" : self.list_accounts_state,
                       "select_account" : self.select_account_state,
                       "credit_debit" : self.credit_debit_state,
                       "destination_name" : self.destination_name_state,
                       "destination_check" : self.check_destination_name_state,
                       "change_destination_name" : self.change_destination_name_state,
                       "api_call" : self.api_call_state,
                       "end_call" : self.end_call_state}
        
        self.priority_states = list()
        self.priority_actions = dict()
        self.templates = templates
        
        self.current_state = self.initial_state
    
    def sort_my_slots(self,slots_given) :
        slots_sorted = list()
        
        if "user_account" in slots_given :
            slots_sorted.append("user_account")
            slots_given.remove("user_account")
        
        if "destination_name" in slots_given :
            slots_sorted.append("destination_name")
            slots_given.remove("destination_name")
        
        if "amount" in slots_given :
            slots_sorted.append("amount")
            slots_given.remove("amount")
        
        for slot in slots_given :
            slots_sorted.append(slot)
        
        return slots_sorted
    
    # store any new values given by the user
    def record_user_values(self,user_action) :
        
        if type(user_action.get_values()) == dict :
            
            for slot,values in user_action.get_values().items() :
                self.user_values[slot] = values
                
     # remove the slots given from the list of slots to ask           
    def remove_informed_slots(self,user_action) :
        
        if user_action.get_slots() :
            for slot in user_action.get_slots() :
                if slot in self.slots_to_ask :
                    self.slots_to_ask.remove(slot)
                
    def speak(self,user_action) :
        
        if user_action == None :
            
            print("user_action received is None")
        
        next_state , bot_action = self.current_state(user_action)
        self.current_state = self.states[next_state]
        
        return bot_action
        
    # meet the initial state, here the user may provide one or more than one values
    # request intent if not given already
    def initial_state(self,user_action) :
        self.record_user_values(user_action)
        self.remove_informed_slots(user_action)
        
        if "intent" in user_action.get_slots() :
            next_state = "list_accounts"
            bot_action = Action(actor="API",
                               action="request_accounts",
                               slots=["name"],
                               values=None,
                               message="accounts:{}".format(self.user_values["name"]),
                               templates=self.templates)
        else :
            next_state = "initial"
            bot_action = Action(actor="Bot",
                               action="request",
                               slots=["intent"],
                               values=None,
                               message="requesting the intent from the user",
                               templates=self.templates)
        return next_state , bot_action
    
    def list_accounts_state(self,user_action) :
        
        if user_action.get_description() == "LIST_OF_SLOTS" :
            next_state = "select_account"
            slot_message = ",".join(user_action.get_slots())
            bot_message = "You have the following accounts : {} , which one do you wish ?".format(slot_message)
            bot_action = Action(actor="Bot",
                                action="request",
                                slots=None,
                                values=None,
                                message=bot_message,
                                description="SELECT_ACCOUNT",
                                templates=self.templates)
        else : 
            next_state = "list_accounts"
            bot_action = Action(actor="API",
                               action="request_accounts",
                               slots=["name"],
                               values=None,
                               message="accounts:{}".format(self.user_values["name"]),
                               templates=self.templates)
            
        return next_state , bot_action
        
    def select_account_state(self,user_action) :
        
        self.record_user_values(user_action)
        self.remove_informed_slots(user_action)
        
        if "user_account" in user_action.get_slots() :
            next_state = "credit_debit"
            bot_action = Action(actor="Bot",
                               action="request",
                               slots=["credit_debit"],
                               values=None,
                               message="requesting credit or debit information",
                               description="credit or debit information",
                               templates=self.templates)
        else :
            next_state = "select_account"
            bot_action = Action(actor="Bot",
                               action="request",
                               slots=None,
                               values=None,
                               message="Please select an account",
                               templates=self.templates)
        
        return next_state , bot_action
    
    def credit_debit_state(self,user_action) :
        
        self.record_user_values(user_action)
        self.remove_informed_slots(user_action)
        
        if "credit_debit" in user_action.get_slots() :
            next_state = "destination_name"
            bot_action = Action(actor="Bot",
                                action="request",
                                slots=["destination_name"],
                                values=None,
                                message="requesting the user to provide the destination name",
                                templates=self.templates)
        else :
            next_state = "credit_debit"
            bot_action = Action(actor="Bot",
                                action="request",
                                slots=["credit_debit"],
                                values=None,
                                message="requesting credit or debit information",
                                templates=self.templates)
        
        return next_state , bot_action
    
    def destination_name_state(self,user_action) :
        
        self.record_user_values(user_action)
        self.remove_informed_slots(user_action)
        
        if "destination_name" in user_action.get_slots() :
            
            next_state = "destination_check"
            bot_action = Action(actor="API",
                                action="destination_name_check",
                                slots=None,
                                values=None,
                                message="destination_name:{}".format(self.user_values["destination_name"]),
                                templates=self.templates)
            
        else :
            next_state = "destination_name"
            bot_action = Action(actor="Bot",
                                action="request",
                                slots=["destination_name"],
                                values=None,
                                message="requesting user to provide destination_name",
                                templates=self.templates)
        
        return next_state , bot_action
            
    def check_destination_name_state(self,user_action) :
        
        self.record_user_values(user_action)
        self.remove_informed_slots(user_action)
        
        if user_action.get_message() == "destination_name_check : success" :
            
            next_state = "api_call"
            api_value = self.user_values["user_account"] + " " + self.user_values["credit_debit"] + " " + self.user_values["destination_name"]
            bot_action = Action(actor="Bot",
                                action="api_call",
                                slots=None,
                                values={"api_call" : api_value},
                                message="API_CALL",
                                description="API_CALL",
                                templates=self.templates)
        else :
            
            next_state = "change_destination_name"
            slot_message = ",".join(self.user_values["destination_names"])
            bot_message = "The recipient you provided is not registered, available list of recipients are : {}".format(slot_message)
            bot_action = Action(actor="Bot",
                                action="request",
                                slots=None,
                                values=None,
                                message=bot_message,
                                description="CHANGE_DESTINATION_NAME",
                                templates=self.templates)
            
        return next_state , bot_action
    
    def change_destination_name_state(self,user_action) :
        
        self.record_user_values(user_action)
        self.remove_informed_slots(user_action)
        
        if user_action.get_message() == "accept" :
            next_state = "destination_name"
            bot_action = Action(actor="Bot",
                                action="request",
                                slots=["destination_name"],
                                values=None,
                                message="requesting user to give new destination name",
                                templates=self.templates)
        else :
            next_state = "end_call"
            bot_action = Action(actor="Bot",
                                action="end_call",
                                slots=None,
                                values=None,
                                message="User denied to change destination name",
                                description="NO_CHANGE_IN_DESTINATION_NAME",
                                templates=self.templates)
        
        return next_state , bot_action
    
    def api_call_state(self,user_action) :
        
        self.record_user_values(user_action)
        self.remove_informed_slots(user_action)
        
        if user_action.get_message().startswith("api_call : success") :
            next_state = "end_call"
            bot_action = Action(actor="Bot",
                                action="end_call",
                               slots=None,
                               values=None,
                               message="api_call successful !!",
                               templates=self.templates)
        else :
            next_state = "end_call"
            bot_action = Action(actor="Bot",
                               action="end_call",
                               slots=None,
                               values=None,
                               message="error in processing request !!",
                               templates=self.templates)
        
        return next_state , bot_action
    
    def end_call_state(self,user_action) :
        print("Reached end of transaction")
        return